In [ ]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import albumentations as A

# Paths
csv_path = "/content/drive/MyDrive/2024/University/FP/csv_files/patches_with_labels.csv"
img_dir = "/content/drive/MyDrive/2024/University/FP/Images/img_patches"
model_save_path = "/content/drive/MyDrive/2024/University/FP/models/skin_lesion_model.keras"

# Load CSV
data_df = pd.read_csv(csv_path)

# Convert the label column to strings for compatibility with ImageDataGenerator
data_df['label'] = data_df['label'].astype(str)

# Function to determine the correct patch folder
def get_patch_folder(patch_id):
    patch_number = patch_id.split('_')[-1].split('.')[0]
    patch_folder = f"Patch_{patch_number}"
    return patch_folder

# Function to construct the full image path
def construct_image_path(row):
    # Determine the main category subfolder based on the label
    if row['label'] == '1':
        main_folder = "mel_patches"
    elif row['label'] == '0':
        main_folder = "bkl_patches"
    else:
        raise ValueError(f"Unknown label for Patch_id: {row['Patch_id']}")

    # Determine the patch folder
    patch_folder = get_patch_folder(row['Patch_id'])

    # Construct the full path
    full_path = os.path.join(img_dir, main_folder, patch_folder, row['Patch_id'])
    
    if not os.path.exists(full_path):
        print(f"Invalid path: {full_path}")

    return full_path

# Apply the function to construct the full paths
data_df['image_path'] = data_df.apply(construct_image_path, axis=1)

# Print the first few file paths for verification
print(data_df['image_path'].head())

# Verify one image path manually
sample_image_path = data_df['image_path'].iloc[0]
print(f"Sample image path: {sample_image_path}")
assert os.path.exists(sample_image_path), f"Image not found at: {sample_image_path}"

# ImageDataGenerator for data augmentation and preprocessing
train_aug = A.Compose([
    A.HorizontalFlip(),
    A.VerticalFlip(),
    A.Rotate(limit=20),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2),
    A.MotionBlur(blur_limit=3),
    A.GaussNoise(var_limit=(10.0, 50.0)),
    A.OpticalDistortion(distort_limit=0.05, shift_limit=0.05),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.20, rotate_limit=20),
    A.RandomCrop(height=64, width=64) 
])

train_generator = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    preprocessing_function=lambda img: train_aug(image=img)['image']
)

validation_generator = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_data_gen = train_generator.flow_from_dataframe(
    dataframe=data_df,
    x_col='image_path',
    y_col='label',
    subset='training',
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode='binary',
    target_size=(64, 64)
)

validation_data_gen = validation_generator.flow_from_dataframe(
    dataframe=data_df,
    x_col='image_path',
    y_col='label',
    subset='validation',
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode='binary',
    target_size=(64, 64)
)

# EfficientNetB0 without Metadata
def build_model(input_shape=(64, 64, 3)):
    # Image Input
    image_input = Input(shape=input_shape)

    # EfficientNet Model
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_tensor=image_input)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(1, activation='sigmoid')(x)

    # Final Model
    model = Model(inputs=image_input, outputs=output)
    return model

model = build_model()

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint(model_save_path, monitor='val_loss', save_best_only=True)

# Train the model
history = model.fit(
    train_data_gen,
    validation_data=validation_data_gen,
    epochs=20,
    callbacks=[early_stop, checkpoint]
)